In [9]:
from dotenv import load_dotenv
import os
load_dotenv()

dbconn = os.getenv("DB_CONN")

In [3]:
pip install "psycopg[binary]" 


Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install psycopg

Note: you may need to restart the kernel to use updated packages.


In [10]:
import psycopg

# establish a connection to the database
conn = psycopg.connect(dbconn)

# or use the connection string directly
# create a cursor object
cur = conn.cursor()

In [26]:
# Create a table for the stock market data
def create_table():
    conn = psycopg.connect(dbconn)
    cur = conn.cursor()
    cur.execute(
        '''
            CREATE TABLE IF NOT EXISTS api_data(
                date TIMESTAMP PRIMARY KEY,
                open FLOAT,
                high FLOAT,
                low FLOAT,
                close FLOAT,
                volume FLOAT
                            );
        '''
    )
    conn.commit()
    cur.close()
    conn.close()

create_table()


In [14]:
# Create a table for the article data
def create_article_table():
    conn = psycopg.connect(dbconn)
    cur = conn.cursor()
    cur.execute(
        '''
            CREATE TABLE IF NOT EXISTS article_data(
                date TIMESTAMP PRIMARY KEY,
                title TEXT,
                author TEXT,
                link TEXT
            );
        '''
    )
    conn.commit()
    cur.close()
    conn.close()

create_article_table()

In [33]:
import pandas as pd

btc_df = pd.read_csv('btc_prices.csv')
btc_df.head()

,symbol,date,open,high,low,close,volume
0,BTC,2025-06-03,92492.40,93037.19,92403.27,92911.79,3.603359
1,BTC,2025-06-02,93129.28,93133.86,90726.84,92494.38,221.004414
2,BTC,2025-06-01,92308.49,93307.12,91577.88,93142.55,95.298677
3,BTC,2025-05-31,91712.80,92599.72,90942.82,92311.30,122.897721
4,BTC,2025-05-30,92825.40,93749.99,91369.44,91725.02,287.208543


In [ ]:
# Insert stock price data to the supabase table

from datetime import datetime
import psycopg
import pandas as pd

def insert_stock_data_from_df(df):
    conn = psycopg.connect(dbconn)
    cur = conn.cursor()

    for _, row in df.iterrows():
        # Convert date to datetime if needed
        if isinstance(row['date'], str):
            date_obj = datetime.strptime(row['date'], "%Y-%m-%d")
        else:
            date_obj = pd.to_datetime(row['date'])

        cur.execute(
            '''
            INSERT INTO api_data (date, open, high, low, close, volume)
            VALUES (%s, %s, %s, %s, %s, %s)
            ON CONFLICT (date) DO NOTHING;
            ''',
            (date_obj, row['open'], row['high'], row['low'], row['close'], row['volume'])
        )

    conn.commit()
    cur.close()
    conn.close()

# Then call it like this:
insert_stock_data_from_df(btc_df)


In [34]:
# Insert article data to the supabase table
def insert_article_data_from_df(df):
    conn = psycopg.connect(dbconn)
    cur = conn.cursor()

    for _, row in df.iterrows():
        # Convert date to datetime if needed
        if isinstance(row['date'], str):
            date_obj = datetime.strptime(row['date'], "%Y-%m-%d")
        else:
            date_obj = pd.to_datetime(row['date'])

        cur.execute(
            '''
            INSERT INTO article_data (date, title, author, link)
            VALUES (%s, %s, %s, %s)
            ON CONFLICT (date) DO NOTHING;
            ''',
            (date_obj, row['title'], row['author'], row['link'])
        )

    conn.commit()
    cur.close()
    conn.close()